In [ ]:
# importing the necessary libraries
import pandas as pd
import numpy as np
import warnings

from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
# to ignore warnings
warnings.filterwarnings('ignore')

# adjust the display settings so that the columns will not get truncated
pd.set_option('display.max_columns', None)

# reading the input
train_data = pd.read_csv('train_data.csv')
test_data = pd.read_csv('test_data.csv')

# let's combine them as it is a hassle to apply every transformation twice
train_data['source'] = 'train'
test_data['source'] = 'test'

In [ ]:
data = pd.concat([train_data, test_data], ignore_index=True)

print(train_data.shape, test_data.shape, data.shape)

# checking out missing values
print(data.isnull().sum())

The 5681 missing values in sales is because test data doesn't have it. So, it is fine.

We need to fill in for item weight and outlet size.

In [ ]:
# let's check if there are any duplicate rows
print(f'No. of rows with and without duplicates: {data.shape[0]}, {data.drop_duplicates().shape[0]}')

In [ ]:
# data description
data.describe(include='all')

**Observations**:

Item and Outler identifier counts are as mentioned in the problem statement (1559, 10).

Item type has 16 unique values. We may need to combine some to simplify things.

In [ ]:
# let's check out the categorical values first
col_name = 'Item_Fat_Content'
data[col_name].value_counts()

In [ ]:
# standardizing fat values
data[col_name] = data[col_name].replace({'LF': 'Low Fat', 'low fat': 'Low Fat', 'reg': 'Regular'})
data[col_name].value_counts()

In [ ]:
data['Item_Type'].value_counts()

In [ ]:
item_id = 'Item_Identifier'
data[item_id].head(50).unique()

Based on values of item identifier, we can see a pattern of FDxxx, DRxxx, NCxxx.

Here xxx is the id factor and first 2 letters being the broad category.

The full form could be food (fruits, snacks, etc.), drinks (soft, hard) and non-consumables (household, health, etc.).

In [ ]:
# hence, making a new item category with these
new_col = 'Item_Category'
data[new_col] = data[item_id].apply(lambda x: x[:2])
data[new_col].value_counts()

There is a catch here. If there is a non consumable category, it doesn't make sense that the fat content column has only 2 unique values.

It should be a data mistake and hence, let's make a separate fat category for this.

In [ ]:
# changing fat content to not applicable when item category is non consumable
data.loc[data[new_col] == 'NC', 'Item_Fat_Content'] = 'NA'
data['Item_Fat_Content'].value_counts()

In [ ]:
col_name = 'Outlet_Size'
data[col_name].value_counts(dropna=False)   # dropna false because we also have nulls here

In [ ]:
# filling in mode value based on outlet type
type_size_mode = data.groupby('Outlet_Type')[col_name].transform(lambda x: x.mode()[0])
data[col_name] = data[col_name].fillna(type_size_mode)
data[col_name].value_counts(dropna=False)   # just to check if there are still nulls present

In [ ]:
# now, let's look at the numerical columns
col_name = 'Item_Weight'
print(data[col_name].min(), data[col_name].max())

# filling up nulls with median weight based on item identifiers
idt_weight_mean = data.groupby(item_id)[col_name].transform('mean')
data[col_name] = data[col_name].fillna(idt_weight_mean)
data[col_name].isnull().sum()   # checking if nulls are still present

In [ ]:
# to turn off the legend globally
plt.rc('legend', frameon=False)

# let's check out the item weight distribution
fig, axes = plt.subplots(1, 2, figsize=(8, 2))
sns.histplot(data, x=col_name, ax=axes[0], kde=True)
sns.boxplot(data, x=col_name, color='#ff7f50', ax=axes[1])
plt.tight_layout()
plt.show()

It doesn't seem like a normal distribution and there are no outliers.

In [ ]:
col_name = 'Item_Visibility'
print(data[col_name].min(), data[col_name].max())

fig, axes = plt.subplots(1, 2, figsize=(8, 2))
sns.histplot(data, x=col_name, color='#ff2052', ax=axes[0], kde=True)
sns.boxplot(data, x=col_name, color='#915c83', ax=axes[1])
plt.tight_layout()
plt.show()

We can see that it is right-skewed, which will have an impact on the models.
Also, there are quite a lot of outliers.

In [ ]:
# applying sqrt to reduce skewness and outliers
new_col = 'Visibility_Sqrt'
data[new_col] = np.sqrt(data[col_name])

fig, axes = plt.subplots(1, 2, figsize=(8, 2))
sns.histplot(data, x=new_col, color='#ff2052', ax=axes[0], kde=True)
sns.boxplot(data, x=new_col, color='#915c83', ax=axes[1])
plt.tight_layout()
plt.show()

In [ ]:
col_name = 'Item_MRP'
print(data[col_name].min(), data[col_name].max())

fig, axes = plt.subplots(1, 2, figsize=(8, 2))
sns.histplot(data, x=col_name, color='#e5202a', ax=axes[0], kde=True)
sns.boxplot(data, x=col_name, color='#760a13', ax=axes[1])
plt.tight_layout()
plt.show()

No outliers and not a normal distribution.

In [ ]:
col_name = 'Outlet_Establishment_Year'
print(data[col_name].unique())

# calculating outlet's age is more meaningful as it can help establish relation between age and sales effectively
# the data is collected in 2013 according to the problem description, hence, using that here for age
new_col = 'Outlet_Age'
data[new_col] = 2013 - data[col_name]
print(data[new_col].unique())

In [ ]:
col_name = 'Item_Outlet_Sales'
print(data[col_name].min(), data[col_name].max())

fig, axes = plt.subplots(1, 2, figsize=(8, 2))
sns.histplot(data, x=col_name, color='#c869f5', ax=axes[0], kde=True)
sns.boxplot(data, x=col_name, color='#f6549c', ax=axes[1])
plt.tight_layout()
plt.show()

The target value is right skewed and has outliers.

In [ ]:
# applying sqrt to reduce skewness and outliers
new_col = 'Sales_Sqrt'
data[new_col] = np.sqrt(data[col_name])

fig, axes = plt.subplots(1, 2, figsize=(8, 2))
sns.histplot(data, x=new_col, color='#c869f5', ax=axes[0], kde=True)
sns.boxplot(data, x=new_col, color='#f6549c', ax=axes[1])
plt.tight_layout()
plt.show()

In [ ]:
# dropping off unnecessary columns
data.drop(columns=['Item_Visibility', 'Item_Type', 'Outlet_Establishment_Year',
            'Item_Outlet_Sales'], inplace=True)

target_col = 'Sales_Sqrt'
data.columns

In [ ]:
# let's make categorical columns numeric
from sklearn.preprocessing import LabelEncoder

# we want original columns for submission
le = LabelEncoder()
data['Item_Id'] = le.fit_transform(data[item_id])   # variable item_id = Item_Identifier
data['Outlet_Id'] = le.fit_transform(data['Outlet_Identifier'])

cat_cols = ['Item_Fat_Content', 'Outlet_Size', 'Outlet_Location_Type',
            'Outlet_Type', 'Item_Category']
data = pd.get_dummies(data, columns=cat_cols, drop_first=True, dtype=np.int8)

In [ ]:
# separate train and test data based on source and dropping unnecessary columns
source_col = 'source'
train_data = data[data[source_col] == 'train'].drop(columns=[item_id, 'Outlet_Identifier', source_col])
test_data = data[data[source_col] == 'test']

res_data = test_data[[item_id, 'Outlet_Identifier']]
test_data.drop(columns=[item_id, 'Outlet_Identifier', source_col, target_col], inplace=True)

In [ ]:
train_data.head(3)

In [ ]:
test_data.head(3)

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import lightgbm as lgb

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error as mse
# rmse is the evaluation metric as per the problem statement

X = train_data.drop(target_col, axis=1)
y = train_data[target_col]

# splitting into training and validation data as there is already separate test data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
models = {'LR': LinearRegression(), 'RR': Ridge(),
          'RFR': RandomForestRegressor(random_state=42),
          'GBR': GradientBoostingRegressor(random_state=42),
          'LGBR': lgb.LGBMRegressor(verbose=-1, random_state=42)}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    rmse = np.sqrt(mse(y_val, y_pred)).round(3)
    print(f'Model: {name:<5}| RMSE: {rmse}')

Gradient Boosting Regressor performed better than the rest.

In [ ]:
gbr_model = models['GBR']
gbr_model.fit(X_train, y_train)

# print the feature importance in decending manner
feat_imp = pd.Series(gbr_model.feature_importances_, index=X_train.columns).sort_values(ascending=False)
print(feat_imp)

In [ ]:
# using shap to check feature importance
import shap

explainer = shap.Explainer(gbr_model, X_train)
shap_values = explainer(X_train)
shap.summary_plot(shap_values, X_train, plot_type='bar')

In [ ]:
# cross validate to check the overall performance
cv_model = GradientBoostingRegressor(random_state=42)
scores = cross_val_score(cv_model, X_train, y_train, scoring='neg_root_mean_squared_error', cv=5)
-scores.mean().round(3)

In [ ]:
from sklearn.ensemble import StackingRegressor

# taking the 3 best performed models above as base
base_models = [('gb', GradientBoostingRegressor(random_state=42)), ('lr', LinearRegression()),
    ('lgb', lgb.LGBMRegressor(verbose=-1, random_state=42))]
meta_model = LinearRegression()

sr = StackingRegressor(estimators=base_models, final_estimator=meta_model)
sr.fit(X_train, y_train)
y_pred = sr.predict(X_val)
rmse = np.sqrt(mse(y_val, y_pred)).round(3)
print('RMSE:', rmse)

In [ ]:
# let's train the best model on the validation data
# and then predict the test data to save the results
sr.fit(X_val, y_val)
y_pred = sr.predict(test_data)

res_data['Item_Outlet_Sales'] = np.square(y_pred)
res_data.to_csv('results.csv', index=False)